In [2]:
from create_placements_stroeer_ssp import transforming_placements_json, transforming_websites_json, merging_placements_websites
from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException
from store.sample import placements
from selenium.webdriver.common.action_chains import ActionChains
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from store.login import login
import time

In [3]:
driver = webdriver.Firefox()
driver.get(login.adscale_url)
time.sleep(10)
driver.find_element_by_id('username').send_keys(login.adscale_user)

ElementNotInteractableException: Message: Element <span id="username"> is not reachable by keyboard


In [12]:
driver.find_element_by_id("username").send_keys("felipe")

ElementNotInteractableException: Message: Element <span id="username"> is not reachable by keyboard


In [2]:
def get_json_data():
    websites_json = []
    placements_json = []
    final_json = []
    placements.sort()
    placements_json = transforming_placements_json(placements)
    websites_json = transforming_websites_json(placements)
    websites_json = merging_placements_websites(websites_json, placements_json)
    return websites_json

In [3]:
def login_stroeer():
    driver = webdriver.Firefox()
    driver.get(login.stroeer_url)
    time.sleep(7)
    driver.find_element_by_name('email').send_keys(login.stroeer_login)
    driver.find_element_by_name('password').send_keys(login.stroeer_pw)
    driver.find_element_by_class_name('auth0-label-submit').click()
    return driver

In [4]:
def checking_websites(driver, websites, helpers):
    for website in websites:
        for helper in helpers:
            if len(website[helper[0]]) > 0:
                driver.find_element_by_id('mat-chip-list-input-0').send_keys(website['website'] + helper[1])
                driver.find_element_by_id('mat-chip-list-input-0').send_keys(Keys.ENTER)
                for x in range(10):
                    try:
                        result = driver.find_element_by_xpath(
                            '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[1]/mat-tab-group/div/'\
                            'mat-tab-body[1]/div/div/pm-media-overview/div/mat-card/mat-card-content/mat-list/mat-list-item/div'\
                            '/div[3]/div[1]/div[2]/h4').text
                        break
                    except StaleElementReferenceException:
                        time.sleep(0.5)
                    except:
                        result = None
                        break
                if result == None:
                    print(f"TEM QUE CRIAR O SITE {website['website']}¡")
                    #tem que criar o site
                    pass
                elif result == website['website'] + helper[1]:
                    website[helper[2]] = 'yes'
                    pass
                else:
                    for i in range(2,30):
                        try:
                            xpath = '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[1]/'\
                                    'mat-tab-group/div/mat-tab-body[1]/div/div/pm-media-overview/div/mat-card/mat-card-content/'\
                                    'mat-list/mat-list-item['+str(i)+']/div/div[3]/div[1]/div[2]/h4'
                            if driver.find_element_by_xpath(xpath).text == website['website'] + helper[1]:
                                website[helper[2]] = 'yes'
                                break
                        except:
                            pass
                delete_field_name(driver)

In [5]:
def checking_placements_ids(driver, placements, helpers):
    driver.find_element_by_id('mat-tab-label-0-1').click()
    for placement in placements:
        for helper in helpers:
            if placement[helper[2]] == 'yes':
                for result in placement[helper[0]]:
                    driver.find_element_by_id('mat-chip-list-input-0').send_keys(result['name'])
                    driver.find_element_by_id('mat-chip-list-input-0').send_keys(Keys.ENTER)
                    driver.find_element_by_id('mat-tab-label-0-1').click()
                    try:
                        platzierung = driver.find_element_by_xpath(
                        '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div/mat-tab-group/'\
                        'div/mat-tab-body[2]/div/div/pm-ad-slots-overview/mat-card/mat-card-content/mat-list/'\
                        'mat-list-item/div/div[3]/div[2]/h4').text
                    except:
                        platzierung = 'no'
                    if platzierung == result['name']:
                        driver.find_element_by_xpath(
                            '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div/mat-tab-group/'\
                            'div/mat-tab-body[2]/div/div/pm-ad-slots-overview/mat-card/mat-card-content/mat-list/'\
                            'mat-list-item[1]/div/pm-tag-generator/button/span'
                                                ).click()
                        result['created'] = get_platzierung_id(driver)
                        closing_window(driver)

                    elif platzierung != 'no':
                         for i in range(2,101):
                            try:
                                platzierung = driver.find_element_by_xpath(
                                    '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div/'\
                                    'mat-tab-group/div/mat-tab-body[2]/div/div/pm-ad-slots-overview/mat-card/'\
                                    'mat-card-content/mat-list/mat-list-item['+str(i)+']/div/div[3]/div[2]/h4'
                                                ).text
                                if platzierung == result['name']:
                                    driver.find_element_by_xpath(
                                        '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div/'\
                                        'mat-tab-group/div/mat-tab-body[2]/div/div/pm-ad-slots-overview/mat-card/'\
                                        'mat-card-content/mat-list/mat-list-item['+str(i)+']/div/pm-tag-generator/'\
                                        'button/span'
                                    ).click()
                                    result['created'] = get_platzierung_id(driver)
                                    closing_window(driver)
                                    break
                            except NoSuchElementException:
                                break
                    delete_field_name(driver)

In [6]:
def delete_field_name(driver):
    time.sleep(0.25)
    driver.find_element_by_xpath(
        '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[1]/ibb-search-bar/div/'\
        'mat-chip-list/div/mat-chip/mat-icon'
    ).click()

In [7]:
def get_platzierung_id(driver):
    time.sleep(1.5)
    for j in range(2000):
        try:
            platz_id = driver.find_element_by_id('mat-input-'+str(j)).text
            platz_id = platz_id[platz_id.find('d=') + 3: platz_id.find('"', platz_id.find('d=') + 3)]
            return platz_id
        except NoSuchElementException:
            pass

In [8]:
def closing_window(driver):
    aux = 0
    for i in range(5000):
        if aux == 1:
            break
        for j in range(600):
            try:
                driver.find_element_by_xpath(
                    '/html/body/div['+str(j)+']/div['+str(i)+']/div/mat-dialog-container/pm-tag-generator-dialog/'\
                    'div[1]/button'
                ).click()
                aux = 1
                break
            except:
                pass

In [9]:
def writing_website(driver, name, helper):
    for i in range(5000):
        try:
            driver.find_element_by_xpath('//*[@id="mat-input-'+str(i)+'"]').send_keys(name + helper[1])
            driver.find_element_by_xpath('//*[@id="mat-input-'+str(i+1)+'"]').send_keys('https://' +name + '/')
            driver.find_element_by_xpath('//*[@id="mat-input-'+str(i+3)+'"]').send_keys(name + '.' +helper[0])
            break
        except:
            pass
    driver.find_element_by_xpath(
    '/html/body/pm-root/div/ng-component/div/div/pm-create-medium/div/div/form/ibb-progress-fab/button/span'
    ).click()
    time.sleep(1)
    for i in range(10):
        try:
            driver.find_element_by_xpath(
                '/html/body/pm-root/div/ng-component/div/div/pm-website-detail/div/pm-ad-slots/'\
                'ibb-no-content-info/div[3]/ibb-no-content-action-tile[1]'
                ).click()
            break
        except:
            time.sleep(0.75) 

In [10]:
def create_website(driver, placements, helpers):
    driver.find_element_by_xpath(
        '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[1]/mat-tab-group/mat-tab-header/'\
        'div[2]/div/div/div[1]/div/span'
        ).click()
    for website in placements:
        for helper in helpers:
            if website[helper[2]] != 'yes' and len(website[helper[0]]) > 0:
                website[helper[2]]='yes'
                for i in range(50):
                    try:
                        driver.find_element_by_xpath(
                            '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[2]/'\
                            'button/span/mat-icon'
                            ).click()
                        break
                    except:
                        time.sleep(1)
                time.sleep(3)
                writing_website(driver, website['website'], helper)
                aux = 0
                for web in website[helper[0]]:
                    aux += 1
                    create_placement(driver, web['name'], web['sizes'])
                    action = ActionChains(driver)
                    for x in range(5):
                        try:
                            button = driver.find_element_by_xpath(
                                        '/html/body/pm-root/div/ng-component/div/div/pm-website-detail/div/pm-ad-slots/'\
                                        'ibb-speed-dial/button'
                            )
                            break
                        except:
                            time.sleep(1)
                    if aux < len(website[helper[0]]):
                        action.move_to_element(button).perform()
                        for x in range(3):
                            try:
                                driver.find_element_by_xpath(
                                    '/html/body/pm-root/div/ng-component/div/div/pm-website-detail/div/pm-ad-slots/'\
                                    'ibb-speed-dial/div/button[1]'
                                    ).click()
                                break
                            except:
                                time.sleep(1)
                web['created'] = scrolling_ids(driver, web['name'])
                driver.find_element_by_xpath(
                            '/html/body/pm-root/div/ng-component/div/pm-website-toolbar/header/div[1]/a'
                                ).click()

In [11]:
def create_placement(driver, name, sizes):
    sizes = sizes.split(',')
    for i in range(10000):
        try:
            driver.find_element_by_id('mat-input-'+str(i)).send_keys(name)
            writing_selecting_sizes(driver, sizes, i+1)
            break
        except:
            pass
    driver.find_element_by_xpath(
        '/html/body/pm-root/div/ng-component/div/div/pm-create-banner-ad-slot/ibb-speed-dial/ibb-progress-fab/'\
        'button/span/mat-icon'
        ).click()

In [12]:
def writing_selecting_sizes(driver, sizes, i):
    for size in sizes:
        aux = 0
        if 'sticky' in size:
            aux = 1
            size = size.replace('sticky', '')
        driver.find_element_by_id('mat-input-'+str(i)).send_keys(size)
        try:
            driver.find_element_by_xpath('/html/body/div[2]/div/div/div/mat-option/span/span').click()
        except:
            driver.find_element_by_id('mat-input-'+str(i)).send_keys(Keys.CONTROL, "a")
            driver.find_element_by_id('mat-input-'+str(i)).send_keys(Keys.BACKSPACE)
        if aux == 1:
            driver.find_element_by_id('mat-input-'+str(i)).send_keys(size + ' sticky')
            try:
                driver.find_element_by_xpath('/html/body/div[2]/div/div/div/mat-option/span/span').click()
            except:
                driver.find_element_by_id('mat-input-'+str(i)).send_keys(Keys.CONTROL, "a")
                driver.find_element_by_id('mat-input-'+str(i)).send_keys(Keys.BACKSPACE)

In [13]:
def scrolling_ids(driver, name):
    time.sleep(1.5)
    pl = ''
    for i in range(3000):
        try:
            platzierung = driver.find_element_by_xpath(
            '/html/body/pm-root/div/ng-component/div/div/pm-website-detail/div/pm-ad-slots/div/pm-list-ad-slot-zones/'\
            'div/mat-card/mat-card-content/mat-list/mat-list-item['+str(i)+']/div/pm-ad-slot-card/div/'\
            'div[1]/pm-banner-ad-slot-card/div/div[1]/h3').text
            if platzierung == name:
                while len(pl) < 1:
                    driver.find_element_by_xpath(
                    '/html/body/pm-root/div/ng-component/div/div/pm-website-detail/div/pm-ad-slots/div/'\
                    'pm-list-ad-slot-zones/div/mat-card/mat-card-content/mat-list/mat-list-item['+str(i)+']'\
                    '/div/pm-ad-slot-card/div/div[2]/pm-tag-generator/button').click()
                    pl_id = get_platzierung_id(driver)
                    closing_window(driver)
                break
        except:
            pass
    return pl_id

In [14]:
placements = get_json_data()
driver = login_stroeer()
time.sleep(10)
helpers = [['tablet', '_t', 'tab_created'],['desktop', '_d', 'desk_created'], ['mobile', '_m', 'mob_created']]

#checking_websites(driver, placements, helpers)
#checking_placements_ids(driver, placements, helpers)
#time.sleep(10)
#create_website(driver, placements, helpers)
#time.sleep(10)
#creating_placements_with_existingwebsite(driver, placements, helpers)

In [17]:
for website in placements:
    for helper in helpers:
        

1001-rimes.com no
1001-rimes.com no
1001-rimes.com no
1mot.net no
1mot.net no
1mot.net no
24sata.hr no
24sata.hr no
24sata.hr no
aachener-nachrichten.de no
aachener-nachrichten.de no
aachener-nachrichten.de no
aachener-zeitung.de no
aachener-zeitung.de no
aachener-zeitung.de no
abendblatt.de no
abendblatt.de no
abendblatt.de no


In [ ]:
for placement in placements:
    for helper in helpers:
        if len(placement[helper[0]]) > 0:
            for result in placement[helper[0]]:
                print(result['name'], result['created'])

In [253]:
for placement in placements:
    for helper in helpers:
        if len(placement[helper[0]]) > 0:
            for result in placement[helper[0]]:
                if 'aachener-nachrichten.de_d_970x250_1' == result['name']:
                    print(len(result['created']))

0


In [205]:
for websites in placements:
    for helper in helpers:
        if len(websites[helper[0]]) > 0 and websites[helper[2]] == 'yes':
            for web in websites[helper[0]]:
                if web['created'] == 'no':
                    print(web['name'])
                    '''
                    for i in range(5000):
                        try:
                            driver.find_element_by_id('mat-chip-list-input-'+str(i)).send_keys(websites['website'] + helper[1])
                            driver.find_element_by_id('mat-chip-list-input-'+str(i)).send_keys(Keys.RETURN)
                            break
                        except:
                            pass
                    time.sleep(1)
                    try:
                        result = driver.find_element_by_xpath(
                                '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[1]/mat-tab-group/div/'\
                                'mat-tab-body[1]/div/div/pm-media-overview/div/mat-card/mat-card-content/mat-list/mat-list-item/div/'\
                                'div[3]/div[1]/div[2]').text
                    except:
                        result = 'no'
                    if result == websites['website'] + helper[1]:
                        driver.find_element_by_xpath(
                                '/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[1]/mat-tab-group/div/'\
                                'mat-tab-body[1]/div/div/pm-media-overview/div/mat-card/mat-card-content/mat-list/mat-list-item/div/'\
                                'div[3]/div[1]/div[2]').click()
                    elif result != 'no' and result != websites['website'] + helper[1]:
                        for i in range(5000):
                            try:
                                xpath = ('/html/body/pm-root/div/ng-component/div/div/pm-portfolio-master-overview/div[1]/mat-tab-group/div/'\
                                    'mat-tab-body[1]/div/div/pm-media-overview/div/mat-card/mat-card-content/mat-list/mat-list-item['+str(i)+']/div/'\
                                    'div[3]/div[1]/div[2]')
                                if websites['website'] + helper[1] == driver.find_element_by_xpath(xpath).text:
                                    driver.find_element_by_xpath(xpath).click()
                                    break
                            except:
                                pass
                    print("analiza ai")
                    for i in range(5):
                        try:
                            driver.find_element_by_xpath(
                                    '/html/body/pm-root/div/ng-component/div/div/pm-website-detail/div/pm-ad-slots/ibb-speed-dial/button'
                                    ).click()
                            driver.find_element_by_xpath(
                                '/html/body/pm-root/div/ng-component/div/div/pm-website-detail/div/pm-ad-slots/ibb-speed-dial/div/button[1]').click()
                            break
                        except:
                            time.sleep(1)
                    create_placement(driver, web['name'], web['sizes'])
                    time.sleep(5)
            for i in range(20):
                try:
                    driver.find_element_by_xpath('//*[@id="mat-tab-label-0-0"]').click()
                except:
                    time.sleep(0.5)
            break
            '''

aachener-nachrichten.de_m_320x250_1


In [213]:
for placement in placements:
    for helper in helpers:
        if len(placement[helper[0]]) > 0 and placement[helper[2]] != 'yes':
            print(placement['name'])

In [214]:
#condición:
for websites in placements:
    for helper in helpers:
        if len(websites[helper[0]]) > 0 and websites[helper[2]] == 'yes':
            for web in websites[helper[0]]:
                if web['created'] == 'no':
                    print(web['name'])

aachener-nachrichten.de_m_320x250_1


In [258]:
pl=''
x = 0
while len(pl) < 1:
    x += 1
    if x > 5:
        pl = 'puta mierda'

In [ ]:
def writing_the_placement_2(driver, name):
    for i in range(10000):
        try:
            driver.find_element_by_id('mat-chip-list-input-'+str(i)).send_keys(name)
            break
        except:
            pass